In [ ]:
# 授權 & 安裝套件
!pip install --quiet gspread google-auth
from google.colab import auth
import gspread
from google.auth import default
import pandas as pd

# 授權帳戶
auth.authenticate_user()
creds, _ = default()
gc = gspread.authorize(creds)

# 使用網址開啟試算表（你要貼自己的 Google Sheet 網址）
spreadsheet = gc.open_by_url("https://docs.google.com/spreadsheets/d/1-wYnr4fy6SlpyV0jXRtFUEmFmfuv5QKXgCVqG6QTDKI/edit?gid=1723544552#gid=1723544552")

# 顯示所有工作表名稱
worksheets = spreadsheet.worksheets()
for ws in worksheets:
    print(ws.title)





麟0522
玄0522
君0522
軒0522
總和0522
麟0527
玄0527
君0527
軒0527
總和0527
加總平均


In [ ]:
# 讀取 "總和0522" 工作表
ws_sum_0522 = spreadsheet.worksheet("總和0522")
df_sum_0522 = pd.DataFrame(ws_sum_0522.get_all_records())

# 讀取 "總和0527" 工作表
ws_sum_0527 = spreadsheet.worksheet("總和0527")
df_sum_0527 = pd.DataFrame(ws_sum_0527.get_all_records())

# 讀取 "加總平均" 工作表
ws_avg = spreadsheet.worksheet("加總平均")
df_avg = pd.DataFrame(ws_avg.get_all_records())


In [ ]:
# 1. 安裝中文字型（只需要執行一次）
!apt-get -y install fonts-noto-cjk

# 2. 使用 matplotlib 的 FontProperties 掛載字型
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
from matplotlib import rcParams
import seaborn as sns
import pandas as pd

# 3. 指定字型路徑（這是 Noto Sans CJK 的實際路徑）
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
my_font = fm.FontProperties(fname=font_path)


Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
fonts-noto-cjk is already the newest version (1:20220127+repack1-1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import pandas as pd
import os
import warnings

# ✅ 忽略所有字型警告訊息
warnings.filterwarnings("ignore")

warnings.filterwarnings("ignore")  # 忽略所有警告
plt.rcParams['font.family'] = 'DejaVu Sans'  # ✅ 英文用這個就不會跳錯了

# ✅ 儲存路徑
save_dir_en = "/content/drive/MyDrive/BPO/電梯熱力圖/英文版"
os.makedirs(save_dir_en, exist_ok=True)

# ✅ 繪圖函式
def draw_heatmap_en(df_raw, title, filename):
    df_clean = df_raw.set_index(df_raw.columns[0])
    df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
    df_clean = df_clean.drop(index=["合計"], errors="ignore")
    df_clean.columns.name = "To Floor"
    df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0).astype(int)

    plt.figure(figsize=(12, 9))
    ax = sns.heatmap(df_clean, annot=True, fmt="d", cmap="Blues", cbar=True)
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    plt.title(title, pad=40, fontproperties=my_font)
    plt.xlabel("To Floor", fontproperties=my_font)
    plt.ylabel("From Floor", fontproperties=my_font)
    plt.xticks(fontproperties=my_font)
    plt.yticks(fontproperties=my_font)
    plt.tight_layout()

    # 儲存圖片
    save_path = f"{save_dir}/{filename}.png"
    plt.savefig(save_path, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"✅ Saved: {save_path}")


# ✅ 產生三張圖
draw_heatmap_en(df_sum_0522, "Elevator Usage Heatmap - 0522", "heatmap_0522")
draw_heatmap_en(df_sum_0527, "Elevator Usage Heatmap - 0527", "heatmap_0527")
draw_heatmap_en(df_avg, "Elevator Usage Heatmap - Avg of Two Days", "heatmap_avg")


✅ Saved: /content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_0522.png
✅ Saved: /content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_0527.png
✅ Saved: /content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_avg.png


In [ ]:
def draw_heatmap_percentage_zh_clean(df_raw, title, filename):
    # 前處理
    df_clean = df_raw.set_index(df_raw.columns[0])
    walk_right_series = df_clean["向右走人數"] if "向右走人數" in df_clean.columns else None
    df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
    df_clean = df_clean.drop(index=["合計"], errors="ignore")
    df_clean.columns.name = "出電梯"
    df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0)

    # 計算百分比（整張圖總和 = 100%）
    total_sum = df_clean.values.sum()
    df_pct = (df_clean / total_sum * 100).round(2)

    # 加入「向右走」欄位（各列右側單獨比例）
    if walk_right_series is not None:
        row_total = df_clean.sum(axis=1)
        walk_right_ratio = (walk_right_series / total_sum * 100).round(2)
        df_pct["向右走"] = walk_right_ratio

    # 繪圖
    fig, ax = plt.subplots(figsize=(12, 9))
    cmap = "Blues"
    sns.heatmap(
        df_pct,
        annot=True,
        fmt=".2f",
        cmap=cmap,
        cbar=True,
        ax=ax,
        linewidths=0,
        linecolor=None,
        square=False
    )

    # 軸設定
    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    # 中文標題與標籤
    fig.suptitle(title, fontproperties=my_font, fontsize=16)
    plt.xlabel("出電梯樓層", fontproperties=my_font)
    plt.ylabel("進電梯樓層", fontproperties=my_font)
    plt.xticks(fontproperties=my_font)
    plt.yticks(fontproperties=my_font)

    # 儲存路徑（路徑應該是「中文版」）
    save_path = f"/content/drive/MyDrive/BPO/電梯熱力圖/中文版/{filename}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"✅ [ZH] 已儲存滿版比例熱圖：{save_path}")

# 中文
draw_heatmap_percentage_zh(df_sum_0522, "0522 電梯人流比例熱力圖 (%)", "heatmap_pct_0522")
draw_heatmap_percentage_zh(df_sum_0527, "0527 電梯人流比例熱力圖 (%)", "heatmap_pct_0527")
draw_heatmap_percentage_zh(df_avg, "兩天平均 電梯人流比例熱力圖 (%)", "heatmap_pct_avg")


✅ [ZH] 已儲存比例熱圖：/content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_pct_0522.png
✅ [ZH] 已儲存比例熱圖：/content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_pct_0527.png
✅ [ZH] 已儲存比例熱圖：/content/drive/MyDrive/BPO/電梯熱力圖/中文版/heatmap_pct_avg.png


In [ ]:
def draw_heatmap_percentage_en(df_raw, title, filename):
    # 前處理
    df_clean = df_raw.set_index(df_raw.columns[0])
    walk_right_series = df_clean["向右走人數"] if "向右走人數" in df_clean.columns else None
    df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
    df_clean = df_clean.drop(index=["合計"], errors="ignore")
    df_clean.columns.name = "To Floor"
    df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0)

    # 計算百分比（整張圖總和 = 100%）
    total_sum = df_clean.values.sum()
    df_pct = (df_clean / total_sum * 100).round(2)

    # 加入 Walk Right 欄位
    if walk_right_series is not None:
        walk_right_ratio = (walk_right_series / total_sum * 100).round(2)
        df_pct["Walk Right"] = walk_right_ratio

    # 繪圖
    fig, ax = plt.subplots(figsize=(12, 9))
    cmap = "Blues"
    sns.heatmap(
        df_pct,
        annot=True,
        fmt=".2f",
        cmap=cmap,
        cbar=True,
        ax=ax,
        linewidths=0,
        linecolor=None,
        square=False
    )

    ax.xaxis.set_ticks_position('top')
    ax.xaxis.set_label_position('top')
    ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

    fig.suptitle(title, fontsize=16)
    plt.xlabel("To Floor")
    plt.ylabel("From Floor")
    plt.xticks()
    plt.yticks()

    save_path = f"/content/drive/MyDrive/BPO/電梯熱力圖/英文版/{filename}.png"
    plt.savefig(save_path, dpi=300, bbox_inches='tight', pad_inches=0)
    plt.close()
    print(f"✅ [EN] Saved full-size percentage heatmap: {save_path}")


# 英文
draw_heatmap_percentage_en(df_sum_0522, "Elevator Usage Percentage - 0522", "heatmap_pct_0522")
draw_heatmap_percentage_en(df_sum_0527, "Elevator Usage Percentage - 0527", "heatmap_pct_0527")
draw_heatmap_percentage_en(df_avg, "Elevator Usage Percentage - Avg of Two Days", "heatmap_pct_avg")

✅ [EN] Saved full-size percentage heatmap: /content/drive/MyDrive/BPO/電梯熱力圖/英文版/heatmap_pct_0522.png
✅ [EN] Saved full-size percentage heatmap: /content/drive/MyDrive/BPO/電梯熱力圖/英文版/heatmap_pct_0527.png
✅ [EN] Saved full-size percentage heatmap: /content/drive/MyDrive/BPO/電梯熱力圖/英文版/heatmap_pct_avg.png


In [ ]:
import os
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 中文字型設定
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
zh_font = fm.FontProperties(fname=font_path)

# 主函式：針對一份 df 產生多張單列圖（中＋英）
def generate_row_percentage_heatmaps(df_raw, dataset_tag):
    # 前處理
    df_clean = df_raw.set_index(df_raw.columns[0])
    walk_right_series = df_clean["向右走人數"] if "向右走人數" in df_clean.columns else None
    df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
    df_clean = df_clean.drop(index=["合計"], errors="ignore")
    df_clean.columns.name = "出電梯"
    df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0)

    # 百分比轉換（橫列總和 = 100）
    df_percentage = df_clean.div(df_clean.sum(axis=1), axis=0) * 100
    df_percentage = df_percentage.fillna(0).round(1)

    # 儲存資料夾
    base_path = "/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖"
    zh_dir = f"{base_path}/中文版/{dataset_tag}"
    en_dir = f"{base_path}/英文版/{dataset_tag}"
    os.makedirs(zh_dir, exist_ok=True)
    os.makedirs(en_dir, exist_ok=True)

    # 遍歷每一樓層
    for idx in df_percentage.index:
        row_data = df_percentage.loc[[idx]]

        # ➕ 加上向右走欄位（若有資料）
        if walk_right_series is not None and idx in walk_right_series.index:
            total = df_clean.loc[idx].sum()
            right_pct = (walk_right_series[idx] / total * 100) if total > 0 else 0
            right_pct = round(right_pct, 1)
            row_data["向右走"] = right_pct

        # 中文圖
        fig, ax = plt.subplots(figsize=(11, 1.5))
        sns.heatmap(row_data, annot=True, fmt=".1f", cmap="Blues", cbar=False, ax=ax)
        ax.set_title(f"從 {idx} 樓出發的出電梯比例（%）", fontproperties=zh_font)
        ax.set_xlabel("出電梯樓層", fontproperties=zh_font)
        ax.set_ylabel("", fontproperties=zh_font)
        ax.set_yticklabels([idx], fontproperties=zh_font)
        ax.set_xticklabels(row_data.columns, fontproperties=zh_font)
        plt.tight_layout()
        zh_path = f"{zh_dir}/row_pct_{idx}.png"
        plt.savefig(zh_path, dpi=300)
        plt.close()
        print(f"✅ [ZH] {dataset_tag} - 儲存：{zh_path}")

        # 英文圖（轉換欄名）
        row_data_en = row_data.copy()
        row_data_en.columns = [col if col != "向右走" else "Walk Right" for col in row_data.columns]

        fig, ax = plt.subplots(figsize=(11, 1.5))
        sns.heatmap(row_data_en, annot=True, fmt=".1f", cmap="Blues", cbar=False, ax=ax)
        ax.set_title(f"Exit Floor Ratio from Floor {idx} (%)", fontsize=12)
        ax.set_xlabel("To Floor")
        ax.set_ylabel("")
        ax.set_yticklabels([f"From {idx}"])
        ax.set_xticklabels(row_data_en.columns)
        plt.tight_layout()
        en_path = f"{en_dir}/row_pct_{idx}.png"
        plt.savefig(en_path, dpi=300)
        plt.close()
        print(f"✅ [EN] {dataset_tag} - Saved: {en_path}")


generate_row_percentage_heatmaps(df_sum_0522, "0522")
generate_row_percentage_heatmaps(df_sum_0527, "0527")
generate_row_percentage_heatmaps(df_avg, "平均")


✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_B1.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_B1.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_1.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_1.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_2.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_2.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_3.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_3.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_4.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_4.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/中文版/0522/row_pct_5.png
✅ [EN] 0522 - Saved: /content/drive/MyDrive/BPO/電梯熱力圖/單列熱圖/英文版/0522/row_pct_5.png
✅ [ZH] 0522 - 儲存：/content/drive/MyDriv

In [ ]:

import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 載入中文字型
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
my_font = fm.FontProperties(fname=font_path)

# 讀取資料
file_path = "/content/BPO-0522 電梯 - 總和.csv"
df_raw = pd.read_csv(file_path)

# 前處理
df_clean = df_raw.set_index(df_raw.columns[0])
df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
df_clean = df_clean.drop(index=["合計"], errors="ignore")
df_clean.columns.name = "出電梯"
df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0)

# ✅ 計算整體比例（全表總人次 = 100%）
total_sum = df_clean.values.sum()
df_global_percentage = df_clean / total_sum * 100
df_global_percentage = df_global_percentage.round(2)

# ✅ 繪製熱力圖（加上 x 軸在上方）
plt.figure(figsize=(12, 9))
ax = sns.heatmap(df_global_percentage, annot=True, fmt=".2f",
                 cmap="YlGnBu", cbar=True, linewidths=0.5, linecolor='gray')

# 把 X 軸移到上方
ax.xaxis.set_ticks_position('top')
ax.xaxis.set_label_position('top')
ax.tick_params(top=True, bottom=False, labeltop=True, labelbottom=False)

# 中文設定
plt.title("全體人流比例熱力圖 (%)", pad=40, fontproperties=my_font)
plt.xlabel("出電梯樓層", fontproperties=my_font)
plt.ylabel("進電梯樓層", fontproperties=my_font)
plt.xticks(fontproperties=my_font)
plt.yticks(fontproperties=my_font)
plt.tight_layout()
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '/content/BPO-0522 電梯 - 總和.csv'

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm

# 載入資料
file_path = "/content/BPO-0522 電梯 - 總和.csv"
df_raw = pd.read_csv(file_path)

# 整理原始表格
df_clean = df_raw.set_index(df_raw.columns[0])
df_clean = df_clean.drop(columns=["向右走人數", "合計"], errors="ignore")
df_clean = df_clean.drop(index=["合計"], errors="ignore")
df_clean.columns.name = "出電梯"
df_clean = df_clean.apply(pd.to_numeric, errors='coerce').fillna(0)

# 轉為百分比形式
df_percentage = df_clean.div(df_clean.sum(axis=1), axis=0) * 100
df_percentage = df_percentage.fillna(0).round(1)

# 載入中文字型
font_path = "/usr/share/fonts/opentype/noto/NotoSansCJK-Regular.ttc"
my_font = fm.FontProperties(fname=font_path)

# 畫圖：每一橫列畫一張熱力圖（共 N 張）
for idx in df_percentage.index:
    row_data = df_percentage.loc[[idx]]  # 選取單一列並保留 DataFrame 格式

    plt.figure(figsize=(10, 1.5))  # 橫向熱力圖
    ax = sns.heatmap(row_data, annot=True, fmt=".1f", cmap="YlGnBu", cbar=False)

    ax.set_title(f"從 {idx} 樓出發的出電梯比例 (%)", fontproperties=my_font)
    ax.set_xlabel("出電梯樓層", fontproperties=my_font)
    ax.set_ylabel("", fontproperties=my_font)
    ax.set_yticklabels([idx], fontproperties=my_font)
    ax.set_xticklabels(row_data.columns, fontproperties=my_font)

    plt.tight_layout()
    plt.show()
